In [1]:
import gcsfs
import cftime
import intake
from cmip6_preprocessing.preprocessing import combined_preprocessing
from fastjmd95 import rho
from xgcm import Grid
import numpy as np
import xesmf as xe
import xarray as xr
import pandas as pd

### Use pandas

In [2]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv', low_memory=False)

In [3]:
uri = df[(df.table_id == 'Omon') & (df.variable_id == 'thetao') & (df.source_id == 'MPI-ESM1-2-HR')
                    & (df.member_id == 'r1i1p1f1') & (df.experiment_id == 'historical')
                    ].zstore.values[0]

In [4]:
gcs = gcsfs.GCSFileSystem(token='anon')
ds = xr.open_zarr(gcs.get_mapper(uri), consolidated=True)
type(ds)

xarray.core.dataset.Dataset

Select specific months

In [8]:
ds = ds.sel(time=ds.time.dt.month.isin([6,7,8]))

Select specific years

In [9]:
plot_data = ds.sel(time=slice('1985', '2014'))

### Use intake-esm

In [10]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6-noQC.json"
col = intake.open_esm_datastore(cat_url)
cat = col.search(table_id='Omon', 
                 experiment_id=['historical'],
                 variable_id=['thetao'],
                 member_id=['r1i1p1f1'],
                 grid_label='gn',
                 source_id=['MPI-ESM1-2-HR']
                )

cmip6_compiled = cat.to_dataset_dict(
            zarr_kwargs={'consolidated':True, 'decode_times':False},
            preprocess=combined_preprocessing
            )


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/Users/odyssey/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/Users/odyssey/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self

In [11]:
ds = cmip6_compiled['CMIP.MPI-M.MPI-ESM1-2-HR.historical.Omon.gn']
type(ds)

xarray.core.dataset.Dataset

In [13]:
ds = ds.sel(time=ds.time.dt.month.isin([6,7,8]))

TypeError: '.dt' accessor only available for DataArray with datetime64 timedelta64 dtype or for arrays containing cftime datetime objects.

In [14]:
plot_data = ds.sel(time=slice('1985', '2014'))